# 1. API KEY 확인

In [ ]:
import os
from openai import OpenAI

# API 키 설정
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")

client = OpenAI(api_key=api_key)

try:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Say this is a test",
            }
        ],
        model="gpt-4o-mini",
    )
    response = chat_completion.choices[0].message.content
    print(response)
except Exception as e:
    print(f"오류가 발생했습니다: {e}")


# 2. 진료데이터 기반 개인화 CRM message 생성

In [ ]:
import pandas as pd

# CSV 파일에서 데이터 읽기
data = pd.read_csv('가상_진료데이터.csv')

# 특정 환자명을 기준으로 필터링
target_patient_name = "환자1"  # 원하는 환자명
filtered_data = data[data['환자명'] == target_patient_name]

# 결과 확인
print(f"필터링된 데이터:\n{filtered_data}")

# 필터링된 데이터로 작업 수행
for _, row in filtered_data.iterrows():
    # 예시: 행별 데이터 접근
    patient_name = row['환자명']
    treatment_code = row['진료코드']
    treatment_description = row['진료코드 설명']
    primary_code = row['주상병코드']
    primary_description = row['주상병코드 설명']

    # 푸시 알림 메시지 생성
    prompt = f"""
당신은 환자를 돌보는 따뜻한 마음을 가진 의사입니다. 아래 환자 데이터를 참고하여, **각기 다른 톤과 목적을 가진 세 가지 메시지**를 작성해주세요. 각 메시지는 자연스러운 사람다운 문체로 작성되어야 하며, 인공지능처럼 보이지 않도록 만들어주세요.

**환자 정보:**
- 환자명: {patient_name}
- 진료코드: {treatment_code}
- 진료명: {treatment_description}
- 주상병코드: {primary_code}
- 주상병코드 설명: {primary_description}

**작성 목표:**
1. 첫 번째 메시지:  
   - 환자를 위로하고 따뜻한 감정을 전달하세요.  
   - 간결한 관리 방법과 친근한 이모티콘을 활용하세요.  
   - 가벼운 일상 대화처럼 느껴지는 문체로 작성하세요.

2. 두 번째 메시지:  
   - 병원 재방문을 유도하는 요소를 포함하세요.  
   - 환자가 치료의 중요성을 느끼도록 하며, 병원을 방문하고 싶게 만드세요.  
   - 약간 더 설득력 있는 톤을 사용하되, 지나치게 광고처럼 느껴지지 않게 하세요.

3. 세 번째 메시지:  
   - 신뢰를 줄 수 있는 전문적인 의학 정보를 제공하세요.  
   - 병명과 치료와 관련된 최신 연구나 효과를 간략히 언급하며 신뢰성을 강조하세요.  
   - 전문성과 따뜻함을 조화롭게 표현하세요.

**출력 형식:**
1. **환자를 위로하는 메시지:**
"안녕하세요, {patient_name}님! 😊  
지난번 진료 이후 상태가 좀 나아지셨는지 궁금해요. 요즘 같은 날씨에 허리 통증은 더 힘들게 느껴질 수 있어요.  
✔️ 하루에 5분 정도 스트레칭으로 근육을 풀어주시면 큰 도움이 될 거예요.  
✔️ 허리가 뻐근할 때 따뜻한 찜질도 잊지 마세요!  
p
혹시 불편함이 계속되시면 언제든 말씀해주세요. {patient_name}님의 건강을 항상 응원합니다! 🌷"

2. **병원 재방문을 유도하는 메시지:**
"{patient_name}님, 안녕하세요!  
지난 진료 후 요통이 얼마나 나아지셨는지 확인하고 싶어요. 😊  
추가적인 상담이나 더 효과적인 맞춤 치료를 원하신다면, 병원에 방문해 주세요! 저희는 {patient_name}님의 상태를 더 자세히 평가하고, 한약 치료 외에도 필요한 도움을 드릴 준비가 되어 있습니다.  

건강한 하루를 위해 작은 변화부터 시작해 보세요. 다음 방문에서 뵙기를 기대합니다! 💊"

3. **전문성을 강조한 메시지:**
"안녕하세요, {patient_name}님. 👩‍⚕️  
이번에 진단받으신 요통은 주로 허리 근육 긴장이나 척추 구조의 문제에서 기인할 수 있습니다. 한약 치료는 염증을 줄이고 혈액 순환을 개선하는 데 큰 효과가 있습니다.  
최근 연구에 따르면 스트레칭과 같은 근육 강화 운동이 요통 관리에 필수적이라는 결과가 있습니다. 특히 고양이 자세와 같은 스트레칭은 허리의 유연성을 높이고 통증을 줄이는 데 도움이 됩니다.  

궁금하신 점이 있거나 추가 치료가 필요하다면 언제든지 병원에 방문해 주세요. 항상 최선을 다해 지원하겠습니다. 😊"
"""
print(f"생성된 프롬프트:\n{prompt}")


try:
    # GPT-4 모델을 사용하여 푸시 알림 제목과 본문 생성
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",  # GPT-4o-mini 모델 사용
    )

    # 모델의 응답에서 제목과 본문을 추출
    response = chat_completion.choices[0].message.content.strip()

    # 결과 출력
    print("푸시 알림 내용:")
    print(response)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")